In [547]:
import torch
import numpy as np

In [548]:
# w = torch.tensor([2.0], requires_grad=True)
# b = torch.tensor([1.0], requires_grad=True)

# # Input
# x = torch.tensor([3.0])

In [549]:
# # Forward pass
# y_pred = w * x + b
# print("y_pred:", {y_pred.item()})

In [550]:
# # Loss
# y_true = torch.tensor([10.0])
# loss = (y_pred - y_true) ** 2
# print("Loss:", {loss.item()})

In [551]:
# print("Avant backward: w.grad =", w.grad)

In [552]:
# loss.backward()

In [553]:
# print("Après backward: w.grad =", w.grad)
# print("Apres backward: b.grad =", b.grad)

In [554]:
# learning_rate = 0.01
# with torch.no_grad(): # Désactiver le tracking des gradients pour l'update w = w + alpha * w
#     w -= learning_rate * w.grad
#     b -= learning_rate * b.grad

In [555]:
# print("Nouveau poids : w =", w, "b =", b)

In [556]:
# # Réinitialiser les gradients
# w.grad.zero_()
# b.grad.zero_()
# print("Après zero_(): w.grad =", w.grad, "b.grad =", b.grad)

In [557]:
import torch.nn as nn

# perceptron = nn.Linear(1, 1)

# # Voir les poids initiaux
# print("Poids :", perceptron.weight)
# print("Biais :", perceptron.bias)

In [558]:
# # Forward
# x = torch.tensor([3.0]) # Shape (batch_size, features)
# print(f"input : {x}")
# y_pred = perceptron(x)
# print(y_pred)


In [559]:
# # Loss
# y_true = torch.tensor([10.0])
# criterion = nn.MSELoss()
# loss = criterion(y_pred, y_true)

In [560]:
# loss

In [561]:
# # Backward
# loss.backward()

In [562]:
# print(f"Gradient de poids : {perceptron.weight.grad}")
# print(f"Gradient de biais : {perceptron.bias.grad}")

In [563]:
import torch.optim as optim
# optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [564]:
# optimizer.step()

In [565]:
# print("Nouveau poids :", perceptron.weight)
# print("Nouveau biais :", perceptron.bias)

In [566]:
# optimizer.zero_grad()

In [567]:
# print(f"Poids après zero_grad : {perceptron.weight.grad}")
# print(f"Biais après zero_grad : {perceptron.bias.grad}")

In [568]:
# class SimpleNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.fc1 = nn.Linear(2, 4)
#         self.fc2 = nn.Linear(4, 3)
#         self.fc3 = nn.Linear(3, 1)
#         self.relu = nn.ReLU()
#         self.sigmoid = nn.Sigmoid()
    
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
#         x = self.sigmoid(x)
#         return x

In [569]:
# model = SimpleNet()

In [570]:
# model

In [571]:
# x = torch.tensor([1.0, 2.0])
# y = torch.tensor([5.0])
# y_pred = model(x)
# loss = nn.MSELoss()(y_pred, y)

# loss.backward()

In [572]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize, RandomHorizontalFlip, RandomCrop, ColorJitter
import matplotlib.pyplot as plt


In [573]:
transform_training = Compose([
    RandomHorizontalFlip(),
    RandomCrop(28, padding=4),
    ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    ToTensor(),
    Normalize((0.1307,),(0.3081,))
])

transform_testing = Compose([
    ToTensor(),
    Normalize((0.1307,),(0.3081,))
])

training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=transform_training
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=transform_testing
)

In [574]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [575]:
# Inspect one transformed image: pixel values and white/black counts
sample_img, sample_label = training_data[0]
print(f"Label: {sample_label}")
img_denorm = sample_img * 0.3081 + 0.1307  # undo normalization back to [0,1] approx
img_np = img_denorm.squeeze().numpy()
print("Pixel array (0..1):")
print(img_np)
white = (img_np >= 0.5).sum()
black = (img_np < 0.5).sum()
print(f"White pixels (>=0.5): {white}, Black pixels (<0.5): {black}, total: {img_np.size}")

Label: 5
Pixel array (0..1):
[[0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628]
 [0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628]
 [0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628 0.01568628 0.01568628
  0.01568628 0.01568628 0.01568628 0.01568628]
 [0.01568628 0

In [576]:
print(training_data.classes)
print(training_data.data.size)

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
<built-in method size of Tensor object at 0x0000020256455430>


In [577]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()
#         self.fc = nn.Sequential(
#             nn.Linear(28*28, 512),
#             nn.ReLU(),
#             nn.Linear(512, 128),
#             nn.ReLU(),
#             nn.Linear(128, 10),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.fc(x)
#         return logits

In [578]:
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
# model = NeuralNetwork().to(device)
# print(model)

In [579]:
# x = torch.rand(1, 28, 28, device=device)
# logits = model(x)

In [580]:
# logits

In [581]:
# pred_probab = nn.Softmax(dim=1)(logits)
# pred_probab

In [582]:
# y_pred = pred_probab.argmax(dim=1)
# y_pred


In [583]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [584]:
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch_idx, batch_value in enumerate(dataloader):
#         X, y = batch_value
#         X, y = X.to(device), y.to(device)

#         # Forward pass
#         pred = model(X)
#         loss = loss_fn(pred, y)

#         # Backward pass
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         if batch_idx % 100 == 0:
#             loss, current = loss.item(), (batch_idx+1) * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5}]")


In [585]:
# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [586]:
# epochs = 10
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(training_dataloader, model, loss_fn, optimizer)
#     test(test_dataloader, model, loss_fn)
# print("Done!")

In [587]:
# example_inputs = (torch.randn(1, 1, 28, 28),)
# model.to('cpu')
# onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)

In [588]:
# onnx_program.save("model.onnx")

In [589]:
import torch.nn.functional as F

In [590]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        self.dropout = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(512 * 7 * 7, 10)  # 28x28 input -> pool -> pool -> conv => 7x7 feature map

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

In [591]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNN().to(device)
print(model)

CNN(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=25088, out_features=10, bias=True)
)


In [592]:
x = torch.rand(1, 1, 28, 28, device=device)
logits = model(x)

In [593]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.0878, 0.1342, 0.1429, 0.0737, 0.0580, 0.0426, 0.1203, 0.1800, 0.0591,
         0.1013]], grad_fn=<SoftmaxBackward0>)

In [594]:
y_pred = pred_probab.argmax(dim=1)
y_pred

tensor([7])

In [595]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [596]:
from torch.utils.tensorboard import SummaryWriter

In [597]:
from datetime import datetime
log_dir = f"runs/mnist_cnn_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
writer = SummaryWriter(log_dir=log_dir)
print(f"TensorBoard logging to {log_dir}")

TensorBoard logging to runs/mnist_cnn_20251211-162230


In [598]:
# writer.add_graph(model, example_inputs)
# writer.flush()

In [599]:
def train(dataloader, model, loss_fn, optimizer, writer, epoch_index):
    size = len(dataloader.dataset)
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    running_seen = 0
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        batch_size = X.size(0)
        running_loss += loss.item() * batch_size
        running_correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        running_seen += batch_size
        global_step = epoch_index * len(dataloader) + batch_idx

        if batch_idx % 10 == 0:
            writer.add_scalar('Training/Batch_Loss', loss.item(), global_step)
            writer.add_scalar('Training/Batch_Accuracy', running_correct / running_seen, global_step)

        if batch_idx % 100 == 0:
            loss_val, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5}]")

    epoch_loss = running_loss / size
    epoch_accuracy = running_correct / size
    writer.add_scalar('Training/Epoch_Loss', epoch_loss, epoch_index)
    writer.add_scalar('Training/Epoch_Accuracy', epoch_accuracy, epoch_index)

In [600]:
def test(dataloader, model, loss_fn, writer, epoch_index):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size

    writer.add_scalar('Test/Accuracy', correct, epoch_index)
    writer.add_scalar('Test/Avg_Loss', test_loss, epoch_index)

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [601]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_dataloader, model, loss_fn, optimizer, writer, t)
    test(test_dataloader, model, loss_fn, writer, t)
writer.flush()
writer.close()
print("Done!")

Epoch 1
-------------------------------


c:\Users\thoma\Documents\IIM\DEEP_LEARNING\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


loss: 2.369785  [   64/60000]
loss: 0.981711  [ 6464/60000]
loss: 0.981711  [ 6464/60000]
loss: 0.750521  [12864/60000]
loss: 0.750521  [12864/60000]
loss: 0.432702  [19264/60000]
loss: 0.432702  [19264/60000]
loss: 0.251398  [25664/60000]
loss: 0.251398  [25664/60000]
loss: 0.300395  [32064/60000]
loss: 0.300395  [32064/60000]
loss: 0.451014  [38464/60000]
loss: 0.451014  [38464/60000]
loss: 0.471761  [44864/60000]
loss: 0.471761  [44864/60000]
loss: 0.460873  [51264/60000]
loss: 0.460873  [51264/60000]
loss: 0.280067  [57664/60000]
loss: 0.280067  [57664/60000]
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.483974 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.483974 

Epoch 2
-------------------------------
loss: 0.687028  [   64/60000]
loss: 0.687028  [   64/60000]
loss: 0.368583  [ 6464/60000]
loss: 0.368583  [ 6464/60000]
loss: 0.199836  [12864/60000]
loss: 0.199836  [12864/60000]
loss: 0.033065  [19264/60000]
loss: 0.033065  [19264/60000]
loss: 

In [602]:
example_inputs = (torch.randn(1, 1, 28, 28),)
model.to('cpu')
onnx_program = torch.onnx.export(model, example_inputs, dynamo=True)

[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 3 of general pattern rewrite rules.
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 3 of general pattern rewrite rules.


In [603]:
onnx_program.save("model.onnx")

In [604]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [605]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 30132), started 5:09:26 ago. (Use '!kill 30132' to kill it.)